In [2]:
import multiprocessing as mp

class Producer(mp.Process):
    def __init__(self,Buffer,BUFFER_SIZE,itemCount,lock):
        super(Producer,self).__init__()
        self.consumidor=None
        self.BUFFER_SIZE=BUFFER_SIZE
        self.Buffer=Buffer
        self.itemCount=itemCount
        self.lock=lock
        self.loop=True
        self.item=0
        
    def addConsimidor(self,consumidor):
        self.consumidor=consumidor
        
    def produceItem(self):
        self.item+=1
        print('Se produce el item:',self.item)
        return self.item

    def sleep(self):
        print("Se duerme el productor")
        self.loop=False
        
    def wakeUp(self):
        print("Se levanta el productor")
        self.loop=True
        self.run()

    def putItemIntoBuffer(self,item):
        self.lock.acquire()
        self.itemCount.value+=1
        self.Buffer.put(item)
        print("Se agrega el item {} al buffer".format(item))
        print("Numero de elementos en el buffer:",self.itemCount.value)
        self.lock.release()

    def run(self):
        while(self.loop):
            item=self.produceItem()
            if self.itemCount.value==self.BUFFER_SIZE:
                self.sleep()
            elif self.itemCount.value == 1:
                self.consumidor.wakeUp()      
            elif self.itemCount.value<self.BUFFER_SIZE:
              self.putItemIntoBuffer(item)

class Consumer(mp.Process):
    def __init__(self,Buffer,BUFFER_SIZE,itemCount,lock):
        super(Consumer,self).__init__()
        self.productor=None
        self.BUFFER_SIZE=BUFFER_SIZE
        self.itemCount=itemCount
        self.lock=lock
        self.Buffer=Buffer
        self.loop=True
       
    def addProductor(self,productor):
        self.productor=productor
        
    def sleep(self):
        print("Se duerme el consumidor")
        self.loop=False
        
    def wakeUp(self):
        print("Se levanta el consumidor")
        self.loop=True
        self.run()
        
    def removeItemFromBuffer(self):
        self.lock.acquire()
        getItem=self.Buffer.get()
        self.itemCount.value-=1
        print("Se remueve el item {} al buffer".format(getItem))
        print("Numero de elementos en el buffer:",self.itemCount.value)
        self.lock.release()
        return getItem

    def consumeItem(self,item):
        print('Se consume el item:',item)

    def run(self):
        while(self.loop):
            if self.itemCount.value==0:
                self.sleep()
            elif self.itemCount.value==(BUFFER_SIZE-1):
                self.productor.wakeUp()                
            elif self.itemCount.value>0:
              item=self.removeItemFromBuffer()
              self.consumeItem(item)



if __name__=='__main__':
    lock=mp.Lock()
    Buffer=mp.Queue()
    BUFFER_SIZE = 4
    itemCount=mp.Value('i',0)
    productor = Producer(Buffer,BUFFER_SIZE,itemCount,lock)
    consumidor=Consumer(Buffer,BUFFER_SIZE,itemCount,lock)
    productor.addConsimidor(consumidor)
    consumidor.addProductor(productor)
    productor.start()
    consumidor.start()
    productor.join()
    consumidor.join()

Se produce el item: 1
Se duerme el consumidor
Se agrega el item 1 al buffer
Numero de elementos en el buffer: 1
Se produce el item: 2
Se levanta el consumidor
Se remueve el item 1 al buffer
Numero de elementos en el buffer: 0
Se consume el item: 1
Se duerme el consumidor
Se produce el item: 3
Se agrega el item 3 al buffer
Numero de elementos en el buffer: 1
Se produce el item: 4
Se levanta el consumidor
Se remueve el item 3 al buffer
Numero de elementos en el buffer: 0
Se consume el item: 3
Se duerme el consumidor
Se produce el item: 5
Se agrega el item 5 al buffer
Numero de elementos en el buffer: 1
Se produce el item: 6
Se levanta el consumidor
Se remueve el item 5 al buffer
Numero de elementos en el buffer: 0
Se consume el item: 5
Se duerme el consumidor
Se produce el item: 7
Se agrega el item 7 al buffer
Numero de elementos en el buffer: 1
Se produce el item: 8
Se levanta el consumidor
Se remueve el item 7 al buffer
Numero de elementos en el buffer: 0
Se consume el item: 7
Se duerm

Process Producer-3:


Se consume el item: 1435


Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "<ipython-input-2-caa3a319f155>", line 45, in run
    self.consumidor.wakeUp()
  File "<ipython-input-2-caa3a319f155>", line 69, in wakeUp
    self.run()
  File "<ipython-input-2-caa3a319f155>", line 91, in run
    self.consumeItem(item)
  File "<ipython-input-2-caa3a319f155>", line 81, in consumeItem
    print('Se consume el item:',item)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py", line 358, in write
    self.flush()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py", line 323, in flush
    self._flush()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py", line 340, in _flush
    parent=self.parent_header, ident=self.topic)
  File "/usr/local/lib/python3.6/dist-packages/jupyter_client/session.py", line 748, in send
    stream.send_multipart(to_send, copy=copy)
  File "/usr/local/lib/pytho

KeyboardInterrupt: ignored

KeyboardInterrupt
